In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, glob
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # GPU
import tensorflow.python as tf

import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from scipy import signal, ndimage
from scipy.io import loadmat, savemat
from skimage import data, img_as_float, exposure
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

In [2]:
from func_processing import img_affine, img_hflip, load, get_y_as_heatmap, get_avg_xy
from sklearn.model_selection import train_test_split

FTRAIN = './data/facial-keypoints-detection/training/training.csv'

In [3]:
# load data
X, Y = load(FTRAIN)
X = X.reshape(-1,96,96)
# split into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [4]:
# training data augmentation
np.random.seed(0)
dz = np.random.randint(-5,5,(X_train.shape[0],2))
X_train0 = []
Y_train0 = []
for i, x in enumerate(X_train):
    xt, yt = img_affine(np.squeeze(x), Y_train[i], dz[i,0], dz[i,1], 96, 96)
    X_train0.append(xt)
    Y_train0.append(yt)
    xf, yf = img_hflip(np.squeeze(x), Y_train[i])
    X_train0.append(xf)
    Y_train0.append(yf)
X_train0 = np.array(X_train0)
Y_train0 = np.array(Y_train0)

X_train0 = np.expand_dims(X_train0, axis=3)

In [5]:
from func_FCN import FCN

y_train0 = get_y_as_heatmap(Y_train0, 96, 96, 5)
X_train, X_valid, y_train, y_valid = train_test_split(X_train0, y_train0, test_size=0.2, random_state=43)

model = FCN(96, 96, 64)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks = [
EarlyStopping(patience=10, verbose=1),
ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
ModelCheckpoint('model-tgs-seg2.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

history = model.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=callbacks, validation_data=(X_valid, y_valid))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3081 samples, validate on 771 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
3072/3081 [============================>.] - ETA: 0s - loss: 0.0064 - acc: 0.2346
Epoch 00001: val_loss improved from inf to 0.00480, saving model to model-tgs-seg2.h5
3081/3081 [==============================] - 9s 3ms/sample - loss: 0.0063 - acc: 0.2348 - val_loss: 0.0048 - val_acc: 0.3143
Epoch 2/50
3072/3081 [============================>.] - ETA: 0s - loss: 0.0045 - acc: 0.3348
Epoch 00002: val_loss improved from 0.00480 to 0.00418, saving model to model-tgs-seg2.h5
3081/3081 [==============================] - 7s 2ms/sample - loss: 0.0045 - acc: 0.3350 - val_loss: 0.0042 - val_acc: 0.3515
Epoch 3/50
3072/3081 [============================>.] - ETA: 0s - loss: 0.0039 - acc: 0.3652
Epoch 00003: val_loss improved from 0.00418 to 0.00379, saving model to model-tg

NameError: name 'sigma' is not defined

In [7]:
plt.plot(history.history['loss'], label='traning loss')
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.yscale('log')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.savefig('./model/hist_fcn.png')
plt.close()

In [6]:
model.save('./model/fcn')